In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-09 22:53:39--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-04-09 22:53:40 (6.55 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|                Y|
|R3LGC3EKEG84PX|          5|            0|          0|   N|                Y|
| R9PYL3OYH55QY|          5|            0|          1|   N|                Y|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|                Y|
|R15LYP3O51UU9E|          5|            0|          0|   N|                Y|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|                Y|
|R32FE8Y45QV434|          5|            0|          0|   N|                Y|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|     

In [6]:
# Step 1: Create a new DF: condition => total_votes>=20
filtered_vine_df = vine_df.filter('total_votes>=20')
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [7]:
# Step 2: Create a new DF: condition => helpful_votes/total_votes >= 50%
helpful_votes50_df = filtered_vine_df.withColumn('grater50', filtered_vine_df['helpful_votes'] / filtered_vine_df['total_votes']).filter('grater50>=0.5')
helpful_votes50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          grater50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|0.9615384615384616|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|0.9615384615384616|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|              0.95|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|0.9047619047619048|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|0.9583333333333334|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|               1.0|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|0.9733333333333334|
| RFN4PNRUD1UW2|          4|  

In [8]:
# Step 3: Create a new DF: condition => vine=='Y'
vine_Y_df = helpful_votes50_df.filter(helpful_votes50_df['vine'] == 'Y')
vine_Y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          grater50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R84VUCDBCI29U|          3|           18|         28|   Y|                N|0.6428571428571429|
| R4V3ICFDTIDIF|          4|           20|         21|   Y|                N|0.9523809523809523|
|R1JZ0JAPW83WFS|          4|           54|         58|   Y|                N|0.9310344827586207|
|R1XH7EA97FAVP7|          3|           35|         44|   Y|                N|0.7954545454545454|
|R1482JAU1ZR7QH|          4|           15|         22|   Y|                N|0.6818181818181818|
|R1GGYGVTHP84DG|          4|           25|         27|   Y|                N|0.9259259259259259|
| RXGU9DSKZJSP0|          3|           21|         22|   Y|                N|0.9545454545454546|
+--------------+-----------+--

In [9]:
# Step 4: Create a new DF: condition => vine=='N'
vine_N_df = helpful_votes50_df.filter(helpful_votes50_df['vine'] == 'N')
vine_N_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          grater50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|0.9615384615384616|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|0.9615384615384616|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|              0.95|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|0.9047619047619048|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|0.9583333333333334|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|               1.0|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|0.9733333333333334|
| RFN4PNRUD1UW2|          4|  

In [10]:
# Step 5.1: vine_reviews_YES: 'total number of reviews' & 'number of 5-stars reviews' & '% of 5-stars reviews'
# Step 5.1.a: total number of reviews
vineY_reviews_TOTAL_df = vine_Y_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)", "Total_Vine_Reviews")
vineY_reviews_TOTAL_df.show()

+------------------+
|Total_Vine_Reviews|
+------------------+
|                 7|
+------------------+



In [11]:
# Step 5.1.b: 5-star revievs 
vineY_reviews_5STAR_df = vine_Y_df.filter('star_rating==5').agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "5-stars_Vine_Reviews")
vineY_reviews_5STAR_df.show()

+--------------------+
|5-stars_Vine_Reviews|
+--------------------+
|                   0|
+--------------------+



In [12]:
vineY_reviews_percentage = vineY_reviews_5STAR_df.collect()[0]['5-stars_Vine_Reviews'] / vineY_reviews_TOTAL_df.collect()[0]['Total_Vine_Reviews'] * 100
round(vineY_reviews_percentage,2)

0.0

In [13]:
# Step 5.2: vine_reviews_NO: 'total number of reviews' & 'number of 5-stars reviews' & '% of 5-stars reviews '
# Step 5.2.a: total number of reviews
vineN_reviews_TOTAL_df = vine_N_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)", "Total_non-Vine_Reviews")
vineN_reviews_TOTAL_df.show()

+----------------------+
|Total_non-Vine_Reviews|
+----------------------+
|                105979|
+----------------------+



In [14]:
# Step 5.2.a: number of 5-stars reviews
vineN_reviews_5STAR_df = vine_N_df.filter('star_rating==5').agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "5-stars_non-Vine_Reviews")
vineN_reviews_5STAR_df.show()

+------------------------+
|5-stars_non-Vine_Reviews|
+------------------------+
|                   67580|
+------------------------+



In [15]:
vineN_reviews_percentage = vineN_reviews_5STAR_df.collect()[0]['5-stars_non-Vine_Reviews'] / vineN_reviews_TOTAL_df.collect()[0]['Total_non-Vine_Reviews'] * 100
round(vineN_reviews_percentage,2)

63.77

In [16]:
# Create data frame for % results
percentage_df = spark.createDataFrame([
                                       (0.0, 63.77)],
                                      ['Percentage Vine Reviews', "Percentage non-Vine Reviews"])
percentage_df.show()

+-----------------------+---------------------------+
|Percentage Vine Reviews|Percentage non-Vine Reviews|
+-----------------------+---------------------------+
|                    0.0|                      63.77|
+-----------------------+---------------------------+



In [17]:
# Positivity Bias - additional calculations
# Vine Reviews
all_stars_YES = vine_Y_df.groupby("star_rating").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Count of Reviews Vine")

In [18]:
# Vine Reviews (cont)
import pyspark.sql.functions as func
all_stars_YES_percentage = all_stars_YES.withColumn('% of total', func.round(all_stars_YES['Count of Reviews Vine']/ \
                                                (vineY_reviews_TOTAL_df.collect()[0]['Total_Vine_Reviews']/100), 2)) \
                                                .orderBy(all_stars_YES['star_rating'].asc()).show()

+-----------+---------------------+----------+
|star_rating|Count of Reviews Vine|% of total|
+-----------+---------------------+----------+
|          3|                    3|     42.86|
|          4|                    4|     57.14|
+-----------+---------------------+----------+



In [19]:
# non-Vine Reviews
all_stars_NO = vine_N_df.groupby("star_rating").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Count of Reviews non-Vine")

In [20]:
# non-Vine Reviews (cont)
all_stars_NO_percentage = all_stars_NO.withColumn('% of total', func.round(all_stars_NO['Count of Reviews non-Vine']/ \
                                            (vineN_reviews_TOTAL_df.collect()[0]['Total_non-Vine_Reviews']/100), 2))  \
                                            .orderBy(all_stars_NO['star_rating'].asc()).show()

+-----------+-------------------------+----------+
|star_rating|Count of Reviews non-Vine|% of total|
+-----------+-------------------------+----------+
|          1|                     9157|      8.64|
|          2|                     5348|      5.05|
|          3|                     7897|      7.45|
|          4|                    15997|     15.09|
|          5|                    67580|     63.77|
+-----------+-------------------------+----------+

